# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [6]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../weather_data/WeatherPy_Database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jumla,NP,29.2747,82.1838,39.16,79,3,4.00,clear sky
1,1,Bredasdorp,ZA,-34.5322,20.0403,63.19,81,54,10.51,light rain
2,2,Langen,DE,49.9933,8.6728,55.56,89,100,4.61,light rain
3,3,Geraldton,AU,-28.7667,114.6000,54.01,82,2,3.44,clear sky
4,4,Punta Arenas,CL,-53.1500,-70.9167,57.31,38,0,31.07,clear sky
...,...,...,...,...,...,...,...,...,...,...
1366,1366,Terrace Bay,CA,48.7834,-87.1000,26.58,71,79,11.25,broken clouds
1367,1367,Santa Maria,BR,-29.6842,-53.8069,81.70,28,0,9.22,clear sky
1368,1368,Krutinka,RU,56.0055,71.5113,14.83,96,100,10.60,overcast clouds
1369,1369,Santa Fe,AR,-31.6333,-60.7000,87.98,31,21,1.99,light rain


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [7]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip?"))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [8]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                    
# Display sample data
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Georgetown,MY,5.4112,100.3354,81.23,88,20,0.00,few clouds
6,6,Mazagao,BR,-0.1150,-51.2894,86.05,66,96,6.24,overcast clouds
13,13,Oranjestad,AW,12.5240,-70.0270,84.90,80,40,17.27,scattered clouds
29,29,Malanville,BJ,11.8685,3.3899,76.42,18,25,7.92,scattered clouds
31,31,Wajir,KE,1.7471,40.0573,79.38,70,21,13.71,light rain
...,...,...,...,...,...,...,...,...,...,...
1360,1360,Mbini,GQ,1.5740,9.6134,76.98,87,100,5.73,moderate rain
1361,1361,Itaituba,BR,-4.2761,-55.9836,86.16,74,40,0.00,scattered clouds
1362,1362,Arraias,BR,-12.9288,-46.9437,75.43,71,96,2.21,light rain
1367,1367,Santa Maria,BR,-29.6842,-53.8069,81.70,28,0,9.22,clear sky


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [9]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Georgetown,MY,5.4112,100.3354,81.23,88,20,0.00,few clouds
6,6,Mazagao,BR,-0.1150,-51.2894,86.05,66,96,6.24,overcast clouds
13,13,Oranjestad,AW,12.5240,-70.0270,84.90,80,40,17.27,scattered clouds
29,29,Malanville,BJ,11.8685,3.3899,76.42,18,25,7.92,scattered clouds
31,31,Wajir,KE,1.7471,40.0573,79.38,70,21,13.71,light rain
...,...,...,...,...,...,...,...,...,...,...
1360,1360,Mbini,GQ,1.5740,9.6134,76.98,87,100,5.73,moderate rain
1361,1361,Itaituba,BR,-4.2761,-55.9836,86.16,74,40,0.00,scattered clouds
1362,1362,Arraias,BR,-12.9288,-46.9437,75.43,71,96,2.21,light rain
1367,1367,Santa Maria,BR,-29.6842,-53.8069,81.70,28,0,9.22,clear sky


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [10]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Display sample data
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng
5,Georgetown,MY,81.23,few clouds,5.4112,100.3354
6,Mazagao,BR,86.05,overcast clouds,-0.1150,-51.2894
13,Oranjestad,AW,84.90,scattered clouds,12.5240,-70.0270
29,Malanville,BJ,76.42,scattered clouds,11.8685,3.3899
31,Wajir,KE,79.38,light rain,1.7471,40.0573
...,...,...,...,...,...,...
1360,Mbini,GQ,76.98,moderate rain,1.5740,9.6134
1361,Itaituba,BR,86.16,scattered clouds,-4.2761,-55.9836
1362,Arraias,BR,75.43,light rain,-12.9288,-46.9437
1367,Santa Maria,BR,81.70,clear sky,-29.6842,-53.8069


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [11]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Georgetown,MY,81.23,few clouds,5.4112,100.3354,
6,Mazagao,BR,86.05,overcast clouds,-0.1150,-51.2894,
13,Oranjestad,AW,84.90,scattered clouds,12.5240,-70.0270,
29,Malanville,BJ,76.42,scattered clouds,11.8685,3.3899,
31,Wajir,KE,79.38,light rain,1.7471,40.0573,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [12]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [13]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
# Display sample data
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Georgetown,MY,81.23,few clouds,5.4112,100.3354,Cititel Penang
6,Mazagao,BR,86.05,overcast clouds,-0.1150,-51.2894,Recanto dos Ferreiras
13,Oranjestad,AW,84.90,scattered clouds,12.5240,-70.0270,Divi Village Golf & Beach Resort
29,Malanville,BJ,76.42,scattered clouds,11.8685,3.3899,Hotel Sejour du Sahel
31,Wajir,KE,79.38,light rain,1.7471,40.0573,Mahatallah Hotel
...,...,...,...,...,...,...,...
1360,Mbini,GQ,76.98,moderate rain,1.5740,9.6134,Candy Vista Mar Mbini
1361,Itaituba,BR,86.16,scattered clouds,-4.2761,-55.9836,Hotel Campos
1362,Arraias,BR,75.43,light rain,-12.9288,-46.9437,Hotel Japão
1367,Santa Maria,BR,81.70,clear sky,-29.6842,-53.8069,Hotel Morotin


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [15]:
# Drop the rows where there is no Hotel Name.

# Import numpy to convert empty strings to nan
import numpy as np

# Replace empty strings in the Hotel Name column with np.nan
hotel_df["Hotel Name"].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)
clean_hotel_df = hotel_df

# Display sample data
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Georgetown,MY,81.23,few clouds,5.4112,100.3354,Cititel Penang
6,Mazagao,BR,86.05,overcast clouds,-0.1150,-51.2894,Recanto dos Ferreiras
13,Oranjestad,AW,84.90,scattered clouds,12.5240,-70.0270,Divi Village Golf & Beach Resort
29,Malanville,BJ,76.42,scattered clouds,11.8685,3.3899,Hotel Sejour du Sahel
31,Wajir,KE,79.38,light rain,1.7471,40.0573,Mahatallah Hotel
...,...,...,...,...,...,...,...
1360,Mbini,GQ,76.98,moderate rain,1.5740,9.6134,Candy Vista Mar Mbini
1361,Itaituba,BR,86.16,scattered clouds,-4.2761,-55.9836,Hotel Campos
1362,Arraias,BR,75.43,light rain,-12.9288,-46.9437,Hotel Japão
1367,Santa Maria,BR,81.70,clear sky,-29.6842,-53.8069,Hotel Morotin


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [16]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [17]:
# Set the file name.
output_data_file = "WeatherPy_Vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [18]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [19]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Cititel Penang</dd>\n<dt>City</dt><dd>Georgetown</dd>\n<dt>Country</dt><dd>MY</dd>\n<dt>Current Weather</dt><dd>few clouds and 81.23 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Recanto dos Ferreiras</dd>\n<dt>City</dt><dd>Mazagao</dd>\n<dt>Country</dt><dd>BR</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 86.05 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Divi Village Golf & Beach Resort</dd>\n<dt>City</dt><dd>Oranjestad</dd>\n<dt>Country</dt><dd>AW</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 84.9 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hotel Sejour du Sahel</dd>\n<dt>City</dt><dd>Malanville</dd>\n<dt>Country</dt><dd>BJ</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 76.42 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Mahatallah Hotel</dd>\n<dt>City</dt><dd>Wajir</dd>\n<dt>Country</dt><dd>KE</dd>\n<dt>Current Weather</dt><dd>light rain and 79.38 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [20]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
5,5.4112,100.3354
6,-0.1150,-51.2894
13,12.5240,-70.0270
29,11.8685,3.3899
31,1.7471,40.0573
32,-0.9500,-80.7333
34,-13.4408,-49.1486
36,-3.8004,102.2655
41,9.4942,50.8122
53,14.8961,-24.4956


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [22]:
# Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create a figure to add the Google map as a layer
fig.add_layer(marker_layer)

# Display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))